In [1]:
import math

from numpy.random import permutation
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

#Leer datos
#Base de datos de Wisconsin Breast Cancer Diagnostic 
#569 muestras,
wbcd = pd.read_csv('wisc_bc_data.csv')


# ver al estructura
#32 propiedades, 1 con un identificador unico, 1 para el diagnóstico, 30 númericas
wbcd.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,87139402,B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,0.05955,0.2360,0.6656,1.670,17.43,0.008045,0.011800,0.01683,0.012410,0.01924,0.002248,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,8910251,B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,0.06491,0.4505,1.1970,3.430,27.10,0.007470,0.035810,0.03354,0.013650,0.03504,0.003318,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,905520,B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,0.06340,0.1967,1.3870,1.342,13.54,0.005158,0.009355,0.01056,0.007483,0.01718,0.002198,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,868871,B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,0.06072,0.3384,1.3430,1.851,26.33,0.011270,0.034980,0.02187,0.019650,0.01580,0.003442,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,9012568,B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,0.05544,0.1783,0.4125,1.338,17.72,0.005012,0.014850,0.01551,0.009155,0.01647,0.001767,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [2]:
# Quitamos la variable id 
# no es necesaria para el clasificador. Si no la quitamos, dará cálculos de distancias irrelevantes
del wbcd['id']
wbcd.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,0.05955,0.2360,0.6656,1.670,17.43,0.008045,0.011800,0.01683,0.012410,0.01924,0.002248,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,0.06491,0.4505,1.1970,3.430,27.10,0.007470,0.035810,0.03354,0.013650,0.03504,0.003318,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,0.06340,0.1967,1.3870,1.342,13.54,0.005158,0.009355,0.01056,0.007483,0.01718,0.002198,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,0.06072,0.3384,1.3430,1.851,26.33,0.011270,0.034980,0.02187,0.019650,0.01580,0.003442,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,0.05544,0.1783,0.4125,1.338,17.72,0.005012,0.014850,0.01551,0.009155,0.01647,0.001767,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [3]:
# Generamos una tabla con los diagnósticos
wbcd.diagnosis.value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [4]:
# recodificamos la colmna diagnosis como factor
# los algoritmos requieren que el valor "objetivo" sea un factor
wbcd.loc[wbcd.diagnosis == 'B', 'diagnosis'] = 0
wbcd.loc[wbcd.diagnosis == 'M', 'diagnosis'] = 1
wbcd['diagnosis'] = wbcd.diagnosis.astype('category')

wbcd.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,0,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,0.05955,0.2360,0.6656,1.670,17.43,0.008045,0.011800,0.01683,0.012410,0.01924,0.002248,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,0,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,0.06491,0.4505,1.1970,3.430,27.10,0.007470,0.035810,0.03354,0.013650,0.03504,0.003318,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,0,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,0.06340,0.1967,1.3870,1.342,13.54,0.005158,0.009355,0.01056,0.007483,0.01718,0.002198,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,0,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,0.06072,0.3384,1.3430,1.851,26.33,0.011270,0.034980,0.02187,0.019650,0.01580,0.003442,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,0,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,0.05544,0.1783,0.4125,1.338,17.72,0.005012,0.014850,0.01551,0.009155,0.01647,0.001767,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [5]:
fact_cols = [
    'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean',
    'compactness_mean',  'concavity_mean', 'points_mean', 'symmetry_mean', 'dimension_mean',
    'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
    'concavity_se', 'points_se', 'symmetry_se', 'dimension_se', 'radius_worst', 'texture_worst',
    'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
    'points_worst', 'symmetry_worst', 'dimension_worst'
]

In [6]:
# Transformamos la tabla a porcentajes
# round(prop.table(table(wbcd$diagnosis)) * 100, digits = 1)
wbcd[fact_cols] = (wbcd[fact_cols] * 100).round(1)

wbcd.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,0,1232.0,1239.0,7885.0,46410.0,10.3,7.0,4.0,3.7,19.6,6.0,23.6,66.6,167.0,1743.0,0.8,1.2,1.7,1.2,1.9,0.2,1350.0,1564.0,8697.0,54910.0,13.8,12.7,12.4,9.4,28.3,6.8
1,0,1060.0,1895.0,6928.0,34640.0,9.7,11.5,6.4,2.6,19.2,6.5,45.1,119.7,343.0,2710.0,0.7,3.6,3.4,1.4,3.5,0.3,1188.0,2294.0,7828.0,42480.0,12.1,25.2,19.2,7.9,29.4,7.6
2,0,1104.0,1683.0,7092.0,37320.0,10.8,7.8,3.0,2.5,17.1,6.3,19.7,138.7,134.2,1354.0,0.5,0.9,1.1,0.7,1.7,0.2,1241.0,2644.0,7993.0,47140.0,13.7,14.8,10.7,7.4,30.0,7.9
3,0,1128.0,1339.0,7300.0,38480.0,11.6,11.4,4.6,4.8,17.7,6.1,33.8,134.3,185.1,2633.0,1.1,3.5,2.2,2.0,1.6,0.3,1192.0,1577.0,7653.0,43400.0,13.7,18.2,8.7,8.6,21.0,6.8
4,0,1519.0,1321.0,9765.0,71180.0,8.0,6.9,3.4,2.7,17.2,5.5,17.8,41.2,133.8,1772.0,0.5,1.5,1.6,0.9,1.6,0.2,1620.0,1573.0,10450.0,81910.0,11.3,17.4,13.6,8.2,24.9,6.8


In [7]:
# Determinamos que se ocupa normalizar los valores
# Funcion para normalizar los valores de 0 a 1
wbcd[fact_cols] = (wbcd[fact_cols] - wbcd[fact_cols].mean()) / (wbcd[fact_cols].max() - wbcd[fact_cols].min())
wbcd.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,0,-0.085536,-0.233333,-0.090657,-0.080929,0.060105,-0.105357,-0.114326,-0.059369,0.074843,-0.059567,-0.061273,-0.121735,-0.056357,-0.042785,0.033271,-0.101418,-0.037648,0.003913,-0.021956,-0.062602,-0.098513,-0.267517,-0.101057,-0.081470,0.037034,-0.123456,-0.118361,-0.070824,-0.013942,-0.104242
1,0,-0.166941,-0.011486,-0.156790,-0.130854,0.005560,0.032680,-0.058120,-0.114096,0.054641,0.046816,0.016597,-0.004386,0.026572,-0.024724,-0.001212,0.079033,0.005281,0.041649,0.203396,-0.028120,-0.156143,-0.072954,-0.144336,-0.112019,-0.074808,-0.002214,-0.064048,-0.122371,0.007711,-0.051954
2,0,-0.146116,-0.083181,-0.145457,-0.119486,0.105560,-0.080817,-0.137745,-0.119071,-0.051419,0.004263,-0.075399,0.037602,-0.071812,-0.050051,-0.070178,-0.123974,-0.052800,-0.090427,-0.050125,-0.062602,-0.137289,0.020330,-0.136118,-0.100566,0.030455,-0.103087,-0.131939,-0.139553,0.019522,-0.032347
3,0,-0.134758,-0.199515,-0.131083,-0.114566,0.178287,0.029613,-0.100275,-0.004643,-0.021116,-0.038290,-0.024330,0.027879,-0.047828,-0.026162,0.136719,0.071514,-0.025022,0.154856,-0.064210,-0.028120,-0.154720,-0.264052,-0.153052,-0.109758,0.030455,-0.070110,-0.147914,-0.098316,-0.157643,-0.104242
4,0,0.050296,-0.205602,0.039258,0.024140,-0.148985,-0.108424,-0.128378,-0.109120,-0.046369,-0.165950,-0.082280,-0.177867,-0.072000,-0.042243,-0.070178,-0.078861,-0.040173,-0.052691,-0.064210,-0.062602,-0.002461,-0.265118,-0.013752,-0.015111,-0.127440,-0.077869,-0.108776,-0.112061,-0.080871,-0.104242


In [8]:
# validamos los datos normalizados
# el valor mínimo debe de ser 0 y el máximo 1 
wbcd[fact_cols].max() - wbcd[fact_cols].min()

radius_mean          1.0
texture_mean         1.0
perimeter_mean       1.0
area_mean            1.0
smoothness_mean      1.0
compactness_mean     1.0
concavity_mean       1.0
points_mean          1.0
symmetry_mean        1.0
dimension_mean       1.0
radius_se            1.0
texture_se           1.0
perimeter_se         1.0
area_se              1.0
smoothness_se        1.0
compactness_se       1.0
concavity_se         1.0
points_se            1.0
symmetry_se          1.0
dimension_se         1.0
radius_worst         1.0
texture_worst        1.0
perimeter_worst      1.0
area_worst           1.0
smoothness_worst     1.0
compactness_worst    1.0
concavity_worst      1.0
points_worst         1.0
symmetry_worst       1.0
dimension_worst      1.0
dtype: float64

In [9]:
# separamos la DB en un set como entrenamiento y otro como prueba
wbcd_test = wbcd.loc[1:469]
wbcd_train = wbcd.loc[470:]

wbcd_test.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,points_se,symmetry_se,dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
1,0,-0.166941,-0.011486,-0.156790,-0.130854,0.005560,0.032680,-0.058120,-0.114096,0.054641,0.046816,0.016597,-0.004386,0.026572,-0.024724,-0.001212,0.079033,0.005281,0.041649,0.203396,-0.028120,-0.156143,-0.072954,-0.144336,-0.112019,-0.074808,-0.002214,-0.064048,-0.122371,0.007711,-0.051954
2,0,-0.146116,-0.083181,-0.145457,-0.119486,0.105560,-0.080817,-0.137745,-0.119071,-0.051419,0.004263,-0.075399,0.037602,-0.071812,-0.050051,-0.070178,-0.123974,-0.052800,-0.090427,-0.050125,-0.062602,-0.137289,0.020330,-0.136118,-0.100566,0.030455,-0.103087,-0.131939,-0.139553,0.019522,-0.032347
3,0,-0.134758,-0.199515,-0.131083,-0.114566,0.178287,0.029613,-0.100275,-0.004643,-0.021116,-0.038290,-0.024330,0.027879,-0.047828,-0.026162,0.136719,0.071514,-0.025022,0.154856,-0.064210,-0.028120,-0.154720,-0.264052,-0.153052,-0.109758,0.030455,-0.070110,-0.147914,-0.098316,-0.157643,-0.104242
4,0,0.050296,-0.205602,0.039258,0.024140,-0.148985,-0.108424,-0.128378,-0.109120,-0.046369,-0.165950,-0.082280,-0.177867,-0.072000,-0.042243,-0.070178,-0.078861,-0.040173,-0.052691,-0.064210,-0.062602,-0.002461,-0.265118,-0.013752,-0.015111,-0.127440,-0.077869,-0.108776,-0.112061,-0.080871,-0.104242
5,0,-0.121032,-0.008443,-0.122791,-0.104004,-0.103531,-0.083884,-0.079197,-0.173797,0.110197,0.004263,-0.043164,0.049315,-0.031101,-0.037425,-0.001212,-0.041267,0.030534,-0.052691,-0.021956,-0.028120,-0.113810,0.034722,-0.103746,-0.088499,-0.048492,-0.058470,-0.012930,-0.163608,0.027396,-0.006203


In [10]:
## ---------- Cargamos los datos a la función knn
# cl = factor de clasificación
# k = número de vecinos a usar 
# - preferentemente usar un número impar para romper empates
# - La raiz cuadrada de las 469 es 21.65, tomamos k = 21
x_columns = fact_cols
y_column = ['diagnosis']

knn = KNeighborsRegressor(n_neighbors=21)
knn.fit(wbcd_train[x_columns], wbcd_train[y_column])

wbcd_predictions = knn.predict(wbcd_test[x_columns])

print(classification_report(wbcd_test, wbcd_predictions))

ValueError: Mix type of y not allowed, got types set(['unknown', 'continuous'])